In [1]:
import shap
from datasets import load_dataset
from src.utils import MODEL_NAME_TO_DESC_DICT, format_text_pred, prepare_text
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoModelForSequenceClassification, pipeline, AutoTokenizer
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm

import lightgbm as lgb
from xgboost import XGBClassifier

/home/james/CodingProjects/TextNTabularExplanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_df = load_dataset('james-burton/imdb_genre_prediction2', split='test')
tab_cols = ['Year','Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)','Metascore', 'Rank']
text_col = ['Description']

test_df_text = prepare_text(test_df, 'text_col_only')
test_df_tab = test_df.to_pandas()[tab_cols]

train_df = load_dataset('james-burton/imdb_genre_prediction2', split='train').to_pandas()
train_df_tab = train_df[tab_cols]
y_train = train_df['Genre_is_Drama']


Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction2-a5449428d75bcc31/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (/home/james/.cache/huggingface/datasets/james-burton___parquet/james-burton--imdb_genre_prediction2-a5449428d75bcc31/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [12]:
shap.kmeans(train_df_tab, 100).data[0]

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


array([2.0140e+03, 1.1000e+02, 6.7000e+00, 3.5003e+04, 2.7360e+01,
       6.1000e+01, 4.1500e+02])

## Tab preds

In [ ]:
tab_model = lgb.LGBMClassifier(random_state=42)
# tab_model = XGBClassifier(random_state=42)
tab_model.fit(train_df_tab,y_train)

def tab_pred_fn(examples):
    preds = tab_model.predict_proba(examples)
    return preds

tab_explainer = shap.KernelExplainer(tab_pred_fn, train_df_tab)
tab_shap_values = tab_explainer.shap_values(test_df_tab[:1])
# tab_explainer_tree = shap.TreeExplainer(tab_model)
# tab_shap_values_tree = tab_explainer_tree.shap_values(test_df_tab)

Using 680 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.
100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


In [5]:
from sklearn.cluster import KMeans

In [7]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(train_df_tab)

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


In [11]:
kmeans.cluster_centers_[0]

array([2.01435294e+03, 1.09823529e+02, 6.73529412e+00, 3.49849412e+04,
       2.73743639e+01, 6.14456831e+01, 4.14705882e+02])

In [16]:
np.bincount(kmeans.labels_).shape

(100,)

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

text = "The quick brown [MASK] jumps over the lazy dog"
encoded = tokenizer.encode(text, add_special_tokens=True)


In [19]:
encoded

[101, 1996, 4248, 2829, 103, 14523, 2058, 1996, 13971, 3899, 102]

In [20]:
tokenizer.decode(encoded)

'[CLS] the quick brown [MASK] jumps over the lazy dog [SEP]'

In [25]:
import scipy.special
import numpy as np
import itertools

def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1))

def shapley_kernel(M,s):
    if s == 0 or s == M:
        return 10000
    return (M-1)/(scipy.special.binom(M,s)*s*(M-s))

def f(X):
    np.random.seed(0)
    beta = np.random.rand(X.shape[-1])
    return np.dot(X,beta) + 10

def kernel_shap(f, x, reference, M):
    X = np.zeros((2**M,M+1))
    X[:,-1] = 1
    weights = np.zeros(2**M)
    V = np.zeros((2**M,M))
    for i in range(2**M):
        V[i,:] = reference

    ws = {}
    for i,s in enumerate(powerset(range(M))):
        s = list(s)
        V[i,s] = x[s]
        X[i,s] = 1
        ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M,len(s))
        weights[i] = shapley_kernel(M,len(s)) # This comes from shap values, where each level gets a specific weight
    y = f(V)
    tmp = np.linalg.inv(np.dot(np.dot(X.T, np.diag(weights)), X))
    return np.dot(tmp, np.dot(np.dot(X.T, np.diag(weights)), y))

M = 4
np.random.seed(1)
x = np.random.randn(M)
reference = np.ones(M)
phi = kernel_shap(f, x, reference, M)
base_value = phi[-1]
shap_values = phi[:-1]

print("  reference =", reference)
print("          x =", x)
print("shap_values =", shap_values)
print(" base_value =", base_value)
print("   sum(phi) =", np.sum(phi))
print("       f(x) =", f(x))

  reference = [1. 1. 1. 1.]
          x = [ 1.62434536 -0.61175641 -0.52817175 -1.07296862]
shap_values = [ 0.34264917 -1.15271105 -0.92112596 -1.12952574]
 base_value = 12.411649429368286
   sum(phi) = 9.550935842132565
       f(x) = 9.55093584213122


In [27]:
import shap
import transformers
# import nlp
import torch
import numpy as np
import scipy as sp

# load a BERT sentiment analysis model
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = transformers.DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
).cuda()

# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)


In [28]:
type(explainer)

shap.explainers._partition.Partition

In [ ]:

# explain the model's predictions on IMDB reviews
imdb_train = nlp.load_dataset("imdb")["train"]
shap_values = explainer(imdb_train[:10], fixed_context=1)